<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH_Markov/blob/main/Markov_Apfel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import random
import sys

r = requests.get("https://raw.githubusercontent.com/ollihansen90/apfelsorten/main/apfelsorten.txt")
data = r.text.split("\n")#[:10]
print(len(data))

def dict_add(dictionary, key, value):
    if key in dictionary.keys():
        if value in dictionary[key].keys():
            dictionary[key][value] += 1
        else:
            dictionary[key][value] = 1
    else:
        dictionary[key] = {">": 0}
        dictionary[key][value] = 1
    return dictionary

def generate_markovdict(data, back=3):
    markov_dict = dict()
    for sorte in data:
        if len(sorte)<max([4,back]):
            continue
        for i in range(back):
            markov_dict = dict_add(markov_dict, (back-i)*"<"+sorte[:i], sorte[i])
        for i in range(back-1, len(sorte)-1):
            markov_dict = dict_add(markov_dict, sorte[i-back+1:i+1], sorte[i+1])
        markov_dict = dict_add(markov_dict, sorte[-back:], ">")
    return markov_dict

class Markov_Apfel():
    def __init__(self, dictionary):
        self.dictionary = dictionary
        self.back = len(list(dictionary.keys())[0])

    def __call__(self):
        key = self.back*"<"
        name = self.back*"<"
        while key!=">":
            next_keys = self.dictionary[name[-self.back:]]
            valliste = []
            keyliste = []
            for dictkey in next_keys.keys():
                valliste.append(next_keys[dictkey])
                keyliste.append(dictkey)
            summe = 0
            cumsum = []
            for eintrag in valliste:
                summe += eintrag
                cumsum.append(summe)
            n = int(random.random()*summe)
            i = 0
            while n>cumsum[i]:
                i += 1
            key = keyliste[i]
            name += key
            #print(key)
        return name.replace(">", "").replace(self.back*"<", "")

In [25]:
######################################################
# Hier kann die Anzahl der Vorgänger bestimmt werden #
n_buchstaben = 1
######################################################

markov_dict = generate_markovdict(data, back=n_buchstaben)
markov_apfel = Markov_Apfel(markov_dict)

print(f"Die Modellgröße beträgt {round(sys.getsizeof(markov_dict)/2**10, ndigits=2)} Kilobyte.")

Die Modellgröße beträgt 2.22 Kilobyte.


In [ ]:
apfellist = [markov_apfel() for _ in range(20)]
for apfel in apfellist:
    if apfel in data:
        print(f"\x1b[31m{apfel}\x1b[31m")
    else:
        print(f"\033[94m{apfel}\033[94m")
